<a href="https://colab.research.google.com/github/samiul272/ColabNotebooks/blob/master/tsdetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy  as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import matplotlib.style  as style
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm  import tqdm
import sklearn as sk
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras import layers, models, callbacks, activations, applications, optimizers, regularizers
from tensorflow.keras.preprocessing import image
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import namedtuple
tf.__version__

'2.0.0'

In [0]:
for i in zip((1,2,3,4),(3,4,5,6)):
    print(i)

(1, 3)
(2, 4)
(3, 5)
(4, 6)


In [0]:
class SignAnnotation:
    def __init__(self, name, n_frame):
        self.name = name
        self.frameAnnotations = []
        self.start_frame = n_frame
        self.end_frame = -1
    def add_annotation(self, n_frame, bounding_box):
        self.frameAnnotations.append((n_frame, bounding_box))
        if(self.end_frame<n_frame):
            self.end_frame = n_frame
    @staticmethod
    def _withinBounds(value, bounds):
        return value >= bounds[0] and value < bounds[1]
    @staticmethod
    def _interp(x1, x2, n_frame, bound):
        alpha = (bound[1] - n_frame)/(bound[1]-bound[0])
        return int(alpha*x1 + (1-alpha)*x2)
    def get_bbox(self, n_frame):
        if n_frame <= self.start_frame or n_frame >= self.end_frame:
            return None
        bounds = [i[0] for i in self.frameAnnotations]
        bounds = zip(bounds[:-1], bounds[1:])
        bboxes = [i[1] for i in self.frameAnnotations]
        bboxes = zip(bboxes[:-1], bboxes[1:])
        for bound, bbox in zip(bounds, bboxes):
            if self._withinBounds(n_frame, bound):
                bbox = [self._interp(i, j, n_frame, bound) for i, j in zip(bbox[0], bbox[1])]
                return (n_frame, bbox)
                

In [0]:
prev_signs = []
all_signs = []
with open('TS2010video.vse','r') as f:
    x = f.readlines()
    for i in x:
        if re.search('\[(.*?)\]', i) != None:
            i = i.replace('\n','').split(':')
            n_frame = int(re.sub('\[|\]|F','',i[0]))
            i = i[-1].split('&')
            sign_name = ''
            curr_signs = []
            for j in i:
                k = j.split('@')
                sign_name = sign_name if k[0] == '' else k[0]
                sign_bounds = k[-1]
                sign_bounds = re.sub('x|y|w|h|=|\(|\)','',sign_bounds).split(',')
                sign_bounds = [int(i) for i in sign_bounds]
                sign = None
                for prev_sign in prev_signs:
                    if prev_sign.name == sign_name:
                        prev_sign.add_annotation(n_frame, sign_bounds)
                        sign = prev_sign
                if sign is None:
                    sign = SignAnnotation(sign_name, n_frame)
                    all_signs.append(sign)
                    sign.add_annotation(n_frame, sign_bounds)
                curr_signs.append(sign)
            prev_signs = curr_signs

In [0]:
def get_signs_in_range(n_frame_start, n_frame_end):
    signs = []
    for sign in all_signs:
        if sign.start_frame > n_frame_start:
            signs.append(sign)
        if sign.start_frame > n_frame_end:
            break
    return signs

[sign.name for sign in get_signs_in_range(550, 570)]

['B31']

In [0]:
list(range(2,8))

[2, 3, 4, 5, 6, 7]